In [55]:
import pandas as pd
import json
import sqlite3

In [56]:
orders = pd.read_csv("/Users/yaswanth/Downloads/orders.csv")

with open("/Users/yaswanth/Downloads/users.json") as f:
    users = pd.DataFrame(json.load(f))

conn = sqlite3.connect(":memory:")

with open("/Users/yaswanth/Downloads/restaurants.sql") as f:
    conn.executescript(f.read())

restaurant = pd.read_sql("SELECT * FROM restaurants", conn)

In [57]:
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [58]:
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [59]:
restaurant.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


# Join

In [60]:
merged = orders.merge(users, on="user_id")
final = merged.merge(restaurant, on="restaurant_id")

df = final.copy()

df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


# Data Cleaning

In [61]:
df.shape

(10000, 12)

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id           10000 non-null  int64  
 1   user_id            10000 non-null  int64  
 2   restaurant_id      10000 non-null  int64  
 3   order_date         10000 non-null  object 
 4   total_amount       10000 non-null  float64
 5   restaurant_name_x  10000 non-null  object 
 6   name               10000 non-null  object 
 7   city               10000 non-null  object 
 8   membership         10000 non-null  object 
 9   restaurant_name_y  10000 non-null  object 
 10  cuisine            10000 non-null  object 
 11  rating             10000 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 937.6+ KB


In [63]:
df.describe()

,order_id,user_id,restaurant_id,total_amount,rating
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1504.117700,251.016700,801.162412,4.045430
std,2886.89568,861.727776,144.622558,405.458753,0.606531
min,1.00000,1.000000,1.000000,100.200000,3.000000
25%,2500.75000,761.000000,127.000000,446.310000,3.500000
50%,5000.50000,1508.000000,251.000000,806.295000,4.100000
75%,7500.25000,2250.250000,376.000000,1149.227500,4.600000
max,10000.00000,3000.000000,500.000000,1499.830000,5.000000


In [64]:
g_orders = df[df["membership"] == "Gold"] # Gold Orders.

len(g_orders)

4987

In [65]:

round(g_orders["total_amount"].mean(), 2)


np.float64(797.15)

In [66]:
# Revenue of the city
df.groupby("city")["total_amount"].sum()

city
Bangalore    2206946.58
Chennai      1990513.03
Hyderabad    1889366.58
Pune         1924797.93
Name: total_amount, dtype: float64

In [67]:
g_orders.groupby("city")["total_amount"].sum()

city
Bangalore     994702.59
Chennai      1080909.79
Hyderabad     896740.19
Pune         1003012.32
Name: total_amount, dtype: float64

In [68]:
df.groupby("cuisine")["total_amount"].mean() # Cuisine

cuisine
Chinese    798.389020
Indian     798.466011
Italian    799.448578
Mexican    808.021344
Name: total_amount, dtype: float64

In [69]:
df.groupby("cuisine")["total_amount"].sum()

cuisine
Chinese    1930504.65
Indian     1971412.58
Italian    2024203.80
Mexican    2085503.09
Name: total_amount, dtype: float64

In [70]:
df["user_id"].nunique()

2883

In [71]:

total = df.groupby("user_id")["total_amount"].sum()

total[total > 1000].count()


np.int64(2544)

In [72]:
len(df[df["rating"] >= 4.5]) # Rating

3374

In [73]:
print(df.columns)
order_count = df.groupby("restaurant_name_x").size()
avg_value = df.groupby("restaurant_name_x")["total_amount"].mean()

stats = pd.concat([order_count, avg_value], axis=1)
stats.columns = ["orders","avg_amount"]

stats[stats["orders"] < 20].sort_values("avg_amount", ascending=False)


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')


,orders,avg_amount
restaurant_name_x,,
Hotel Dhaba Multicuisine,13,1040.222308
Sri Mess Punjabi,12,1029.180833
Ruchi Biryani Punjabi,16,1002.140625
Sri Delights Pure Veg,18,989.467222
Classic Kitchen Family Restaurant,19,973.167895
...,...,...
Annapurna Tiffins Punjabi,19,621.828947
Darbar Tiffins Non-Veg,18,596.815556
Darbar Restaurant Punjabi,14,589.972857


In [74]:
df.groupby(["membership","cuisine"])["total_amount"].sum() # Combine using groupby

membership  cuisine
Gold        Chinese     977713.74
            Indian      979312.31
            Italian    1005779.05
            Mexican    1012559.79
Regular     Chinese     952790.91
            Indian      992100.27
            Italian    1018424.75
            Mexican    1072943.30
Name: total_amount, dtype: float64

In [75]:
# Time Analysis
df["order_date"] = pd.to_datetime(df["order_date"])

df["quarter"] = df["order_date"].dt.to_period("Q")

df.groupby("quarter")["total_amount"].sum()

/var/folders/_p/7k2_x2f915v9t7cvhmx97s2r0000gn/T/ipykernel_72879/1258927315.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


quarter
2023Q1    1993425.14
2023Q2    1945348.72
2023Q3    2037385.10
2023Q4    2018263.66
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

In [76]:
len(g_orders)

4987

In [77]:
df["user_id"].nunique()

2883

In [78]:
g_orders.groupby("city")["total_amount"].sum().idxmax()

'Chennai'

In [79]:
df.groupby("cuisine")["total_amount"].mean().idxmax()

'Mexican'